In [1]:
import pandas as pd
import numpy as np

import geopandas as gpd
from shapely.geometry import Point
from sklearn.neighbors import BallTree

In [2]:
accident_data_with_vehicle_data_futher_cleanedd = pd.read_csv("../../data/accident_data_with_vehicle_data_futher_cleanedd.csv",low_memory=True)
df = accident_data_with_vehicle_data_futher_cleanedd
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.columns

Index(['Index', 'Longitude', 'Latitude', 'Accident_Severity', 'Day_of_Week',
       'Road_Type', 'Speed_limit', 'Road_Surface_Conditions',
       'Urban_or_Rural_Area', 'Year', 'Hour', 'Day_of_Year', 'Cluster_1',
       'Vehicle_Type', 'Sex_of_Driver', 'Age_of_Driver',
       'Engine_Capacity_(CC)', 'Age_of_Vehicle', 'Driver_Home_Area_Type'],
      dtype='object')

In [26]:
df.shape

(684637, 19)

In [27]:
stations = pd.read_csv('./weather_data_gather/stations.csv') 
stations['name'] = stations['name'].apply(lambda x: x[:-9])
stations.drop_duplicates(inplace=True,ignore_index=True)
stations

,name,lat,log
0,aberdeenshire_aberdeen-mannofield-resr,57.128,-2.140
1,aberdeenshire_aboyne-no-2,57.076,-2.841
2,aberdeenshire_balmoral,57.037,-3.220
3,aberdeenshire_braemar,57.006,-3.398
4,aberdeenshire_braemar-no-2,57.011,-3.397
...,...,...,...
561,wiltshire_boscombe-down,51.162,-1.755
562,wiltshire_boyton,51.162,-2.070
563,wiltshire_lacock,51.431,-2.115
564,wiltshire_larkhill,51.202,-1.806


In [28]:
df.head()

,Index,Longitude,Latitude,Accident_Severity,Day_of_Week,Road_Type,Speed_limit,Road_Surface_Conditions,Urban_or_Rural_Area,Year,Hour,Day_of_Year,Cluster_1,Vehicle_Type,Sex_of_Driver,Age_of_Driver,Engine_Capacity_(CC),Age_of_Vehicle,Driver_Home_Area_Type
0,200501BS000011,-0.191170,51.489096,2.0,3.0,Single carriageway,30.0,Wet/Damp,1.0,2005.0,17.0,4.0,0.0,9.0,2.0,74.0,-1.0,-1.0,1.0
1,200501BS000021,-0.211708,51.520075,3.0,4.0,Dual carriageway,30.0,Dry,1.0,2005.0,17.0,5.0,1.0,11.0,1.0,42.0,8268.0,3.0,-1.0
2,200501BS000041,-0.173862,51.482442,3.0,6.0,Single carriageway,30.0,Dry,1.0,2005.0,10.0,7.0,2.0,9.0,2.0,49.0,1769.0,4.0,1.0
3,200501BS000091,-0.187623,51.502260,3.0,6.0,Dual carriageway,30.0,Dry,1.0,2005.0,17.0,14.0,3.0,9.0,1.0,68.0,5343.0,16.0,1.0
4,200501BS000101,-0.167342,51.483420,3.0,7.0,Single carriageway,30.0,Dry,1.0,2005.0,22.0,15.0,4.0,9.0,1.0,35.0,1998.0,13.0,1.0


In [29]:
gdf_df = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

gdf_stations = gpd.GeoDataFrame(
    stations, geometry=gpd.points_from_xy(stations.log, stations.lat))

In [30]:
gdf_df.head(5)['geometry']

0    POINT (-0.19117 51.48910)
1    POINT (-0.21171 51.52007)
2    POINT (-0.17386 51.48244)
3    POINT (-0.18762 51.50226)
4    POINT (-0.16734 51.48342)
Name: geometry, dtype: geometry

In [31]:
gdf_stations.head(5)['geometry']

0    POINT (-2.14000 57.12800)
1    POINT (-2.84100 57.07600)
2    POINT (-3.22000 57.03700)
3    POINT (-3.39800 57.00600)
4    POINT (-3.39700 57.01100)
Name: geometry, dtype: geometry

In [32]:
# Create a BallTree 
tree = BallTree(gdf_stations[['lat', 'log']].values, leaf_size=2 )

gdf_df['distance_nearest'], gdf_df['Station ID'] = tree.query(
    gdf_df[['Latitude', 'Longitude']].values, # The input array for the query
    k=1, # The number of nearest neighbors
)

In [33]:
gdf_df.head(5)

,Index,Longitude,Latitude,Accident_Severity,Day_of_Week,Road_Type,Speed_limit,Road_Surface_Conditions,Urban_or_Rural_Area,Year,...,Cluster_1,Vehicle_Type,Sex_of_Driver,Age_of_Driver,Engine_Capacity_(CC),Age_of_Vehicle,Driver_Home_Area_Type,geometry,distance_nearest,Station ID
0,200501BS000011,-0.191170,51.489096,2.0,3.0,Single carriageway,30.0,Wet/Damp,1.0,2005.0,...,0.0,9.0,2.0,74.0,-1.0,-1.0,1.0,POINT (-0.19117 51.48910),0.062236,230
1,200501BS000021,-0.211708,51.520075,3.0,4.0,Dual carriageway,30.0,Dry,1.0,2005.0,...,1.0,11.0,1.0,42.0,8268.0,3.0,-1.0,POINT (-0.21171 51.52007),0.051771,225
2,200501BS000041,-0.173862,51.482442,3.0,6.0,Single carriageway,30.0,Dry,1.0,2005.0,...,2.0,9.0,2.0,49.0,1769.0,4.0,1.0,POINT (-0.17386 51.48244),0.048436,230
3,200501BS000091,-0.187623,51.502260,3.0,6.0,Dual carriageway,30.0,Dry,1.0,2005.0,...,3.0,9.0,1.0,68.0,5343.0,16.0,1.0,POINT (-0.18762 51.50226),0.056689,230
4,200501BS000101,-0.167342,51.483420,3.0,7.0,Single carriageway,30.0,Dry,1.0,2005.0,...,4.0,9.0,1.0,35.0,1998.0,13.0,1.0,POINT (-0.16734 51.48342),0.042266,230


In [35]:
gdf_df.iloc[gdf_df['Station ID'].argmax(), 0:40]
gdf_df['Station ID'] = gdf_df['Station ID'].apply(lambda x: gdf_stations['name'].iloc[x])

In [39]:
df1 = pd.DataFrame(gdf_df.drop(columns=['geometry','distance_nearest']))
df1.head(5)

,Index,Longitude,Latitude,Accident_Severity,Day_of_Week,Road_Type,Speed_limit,Road_Surface_Conditions,Urban_or_Rural_Area,Year,Hour,Day_of_Year,Cluster_1,Vehicle_Type,Sex_of_Driver,Age_of_Driver,Engine_Capacity_(CC),Age_of_Vehicle,Driver_Home_Area_Type,Station ID
0,200501BS000011,-0.191170,51.489096,2.0,3.0,Single carriageway,30.0,Wet/Damp,1.0,2005.0,17.0,4.0,0.0,9.0,2.0,74.0,-1.0,-1.0,1.0,greater-london_london-st-jamess-park
1,200501BS000021,-0.211708,51.520075,3.0,4.0,Dual carriageway,30.0,Dry,1.0,2005.0,17.0,5.0,1.0,11.0,1.0,42.0,8268.0,3.0,-1.0,greater-london_hampstead
2,200501BS000041,-0.173862,51.482442,3.0,6.0,Single carriageway,30.0,Dry,1.0,2005.0,10.0,7.0,2.0,9.0,2.0,49.0,1769.0,4.0,1.0,greater-london_london-st-jamess-park
3,200501BS000091,-0.187623,51.502260,3.0,6.0,Dual carriageway,30.0,Dry,1.0,2005.0,17.0,14.0,3.0,9.0,1.0,68.0,5343.0,16.0,1.0,greater-london_london-st-jamess-park
4,200501BS000101,-0.167342,51.483420,3.0,7.0,Single carriageway,30.0,Dry,1.0,2005.0,22.0,15.0,4.0,9.0,1.0,35.0,1998.0,13.0,1.0,greater-london_london-st-jamess-park


In [41]:
df1.to_csv("../../data/accident_data_with_vehicle_data_further_cleaned_w_stations.csv") 